### Extract Coordinates from Gaussian Logs and Generate ECD GJF Files

This section processes Gaussian geometry optimization output (`.log`) files to generate new Gaussian input (`.gjf`) files for **Electronic Circular Dichroism (ECD)** calculations.  
The steps are as follows:

1. Read `.log` files from the `opt_log/` directory
2. Extract atomic coordinates from the last "Standard orientation" block
3. Replace atomic numbers with element symbols using a predefined dictionary
4. Write the coordinates and Gaussian headers into a new `.gjf` file in the `ecd_gjf/` directory

> Each `.gjf` file is formatted for TD-DFT ECD calculations with CAM-B3LYP/6-31G(d) and 20 excited states.

In [3]:
import os

# Define the input and output directory paths.
input_folder = "opt_log"# The directory path for storing Gaussian-optimized log files.
output_folder = "ecd_gjf" # The directory path for storing the generated GJF files.

# The mapping between element symbols and atomic numbers.
atomic_symbols = atomic_symbols = {
    1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 10: 'Ne',
    11: 'Na', 12: 'Mg', 13: 'Al', 14: 'Si', 15: 'P', 16: 'S', 17: 'Cl', 18: 'Ar', 19: 'K',
    20: 'Ca', 21: 'Sc', 22: 'Ti', 23: 'V', 24: 'Cr', 25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni',
    29: 'Cu', 30: 'Zn', 31: 'Ga', 32: 'Ge', 33: 'As', 34: 'Se', 35: 'Br', 36: 'Kr', 37: 'Rb',
    38: 'Sr', 39: 'Y', 40: 'Zr', 41: 'Nb', 42: 'Mo', 43: 'Tc', 44: 'Ru', 45: 'Rh', 46: 'Pd',
    47: 'Ag', 48: 'Cd', 49: 'In', 50: 'Sn', 51: 'Sb', 52: 'Te', 53: 'I', 54: 'Xe', 55: 'Cs',
    56: 'Ba', 57: 'La', 58: 'Ce', 59: 'Pr', 60: 'Nd', 61: 'Pm', 62: 'Sm', 63: 'Eu', 64: 'Gd',
    65: 'Tb', 66: 'Dy', 67: 'Ho', 68: 'Er', 69: 'Tm', 70: 'Yb', 71: 'Lu', 72: 'Hf', 73: 'Ta',
    74: 'W', 75: 'Re', 76: 'Os', 77: 'Ir', 78: 'Pt', 79: 'Au', 80: 'Hg', 81: 'Tl', 82: 'Pb',
    83: 'Bi', 84: 'Po', 85: 'At', 86: 'Rn', 87: 'Fr', 88: 'Ra', 89: 'Ac'}

# Ensure the output folder exists.
os.makedirs(output_folder, exist_ok=True)

# Iterate through all files in the log folder.
for filename in os.listdir(input_folder):
    if filename.endswith('.log'):  # Ensure that only log files are processed.
        # Construct the full paths for the input and output files.
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename.replace('.log', '_ECD.gjf'))

        # Read the log file.
        with open(input_file, 'r') as f:
            log_lines = f.readlines()

        # Find the index of the last occurrence of 'Standard orientation:'.
        last_occurrence_index = None
        for i, line in enumerate(log_lines):
            if line.strip() == 'Standard orientation:':
                last_occurrence_index = i

        if last_occurrence_index is not None:
            # Extract molecular information.
            molecule_lines = log_lines[last_occurrence_index+5:]
            molecule_info = ''

            for mol_line in molecule_lines:
                parts = mol_line.strip().split()
                if len(parts) < 6:
                    continue
                atom_num = parts[1]
                x, y, z = parts[3:6]
                atom_line = f'{atom_num:<3} {x:>10} {y:>10} {z:>10}\n'
                molecule_info += atom_line

            # Delete the 'constants' line and everything after it.
            if 'constants' in molecule_info:
                constants_index = molecule_info.index('constants')
                molecule_info = molecule_info[:constants_index]

            # Generate the content of the .gjf file.
            gjf_content = f'%chk={filename.replace(".log", "_ECD.chk")}\n%mem=10GB\n# td=(nstates=20,root=20) cam-b3lyp/6-31g(d)\n\nTitle\n\n0 1\n{molecule_info}\n\n'

            # Write to the .gjf file.
            with open(output_file, 'w') as f:
                f.write(gjf_content)

            print(f'Generated {output_file}')

        # Read the generated .gjf file and perform element symbol replacement.
        with open(output_file, 'r') as f:
            gjf_lines = f.readlines()

        modified_gjf_lines = []
        for gjf_line in gjf_lines:
            parts = gjf_line.strip().split()
            if len(parts) < 4:
                modified_gjf_lines.append(gjf_line)
                continue
            atom_num = int(parts[0])
            atom_symbol = atomic_symbols.get(atom_num, '')

            # Generate the modified .gjf file content.
            modified_gjf_line = f' {atom_symbol:<3}                  {parts[1]:>10}   {parts[2]:>10}   {parts[3]:>10}\n'
            modified_gjf_lines.append(modified_gjf_line)

        # Write the modified .gjf file.
        with open(output_file, 'w') as f:
            f.writelines(modified_gjf_lines)

        print(f'Modified {output_file}')

print('Batch gjf modification completed!')

Generated ecd_gjf\molecule_1_ECD.gjf
Modified ecd_gjf\molecule_1_ECD.gjf
Generated ecd_gjf\molecule_10_ECD.gjf
Modified ecd_gjf\molecule_10_ECD.gjf
Generated ecd_gjf\molecule_2_ECD.gjf
Modified ecd_gjf\molecule_2_ECD.gjf
Generated ecd_gjf\molecule_3_ECD.gjf
Modified ecd_gjf\molecule_3_ECD.gjf
Generated ecd_gjf\molecule_4_ECD.gjf
Modified ecd_gjf\molecule_4_ECD.gjf
Generated ecd_gjf\molecule_5_ECD.gjf
Modified ecd_gjf\molecule_5_ECD.gjf
Generated ecd_gjf\molecule_6_ECD.gjf
Modified ecd_gjf\molecule_6_ECD.gjf
Generated ecd_gjf\molecule_7_ECD.gjf
Modified ecd_gjf\molecule_7_ECD.gjf
Generated ecd_gjf\molecule_8_ECD.gjf
Modified ecd_gjf\molecule_8_ECD.gjf
Generated ecd_gjf\molecule_9_ECD.gjf
Modified ecd_gjf\molecule_9_ECD.gjf
Batch gjf modification completed!
